# Using Embedding Projector

Using the TensorBoard Embedding Projector, you can graphically represent high dimensional embeddings. This can be helpful in visualizing, examining, and understanding your embedding layers.

In [0]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# IMDB Data

We will be using a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word. Later in the tutorial, we will be removing this row from the visualization.

In [0]:
(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
encoder = info.features["text"].encoder

In [0]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

In [0]:
train_batch, train_labels = next(iter(train_batches))

# Create Embeddings

A Keras Embedding Layer can be used to train an embedding for each word in your volcabulary. Each word (or sub-word in this case) will be associated with a 16-dimensional vector (or embedding) that will be trained by the model.

See this tutorial to learn more about word embeddings.

In [0]:
# Create an embedding layer
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)

model = tf.keras.Sequential([
                             embedding,
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(16, activation="relu"),
                             tf.keras.layers.Dense(1),

])

In [10]:
# Compile model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model
history = model.fit(
    train_batches, epochs=10, validation_data=test_batches, validation_steps=20
)

Epoch 1/10
2500/2500 [==============================] - 14s 5ms/step - loss: 0.5206 - accuracy: 0.6832 - val_loss: 0.3595 - val_accuracy: 0.8400
Epoch 2/10
2500/2500 [==============================] - 14s 5ms/step - loss: 0.2895 - accuracy: 0.8821 - val_loss: 0.2673 - val_accuracy: 0.8950
Epoch 3/10
2500/2500 [==============================] - 14s 5ms/step - loss: 0.2325 - accuracy: 0.9082 - val_loss: 0.3174 - val_accuracy: 0.8500
Epoch 4/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.2002 - accuracy: 0.9226 - val_loss: 0.2599 - val_accuracy: 0.9050
Epoch 5/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1780 - accuracy: 0.9342 - val_loss: 0.3430 - val_accuracy: 0.8600
Epoch 6/10
2500/2500 [==============================] - 14s 5ms/step - loss: 0.1592 - accuracy: 0.9409 - val_loss: 0.4788 - val_accuracy: 0.8150
Epoch 7/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1435 - accuracy: 0.9483 - val_loss: 0.6057 - val_ac

# Saving data for TensorBoard

TensorBoard reads tensors and metadata from your tensorflow projects from the logs in the specified log_dir directory. For this tutorial, we will be using /logs/imdb-example/.

In order to visualize this data, we will be saving a checkpoint to that directory, along with metadata to understand which layer to visualize.

In [0]:
# Set up a logs directory, so Tensorboard knows where to look for files
log_dir='/logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown"
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))

In [0]:
# Save the weights we want to analyse as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, so
# we will remove that value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])

In [13]:
# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'/logs/imdb-example/embedding.ckpt-1'

In [0]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [0]:
%tensorboard --logdir /logs/imdb-example/

# Analysis



The TensorBoard Projector is a great tool for analyzing your data and seeing embedding values relative to each other. 

The dashboard allows searching for specific terms, and highlights words that are nearby in the embedding space. 

From this example we can see that Wes Anderson and Alfred Hitchcock are both rather neutral terms, but that they are referenced in different contexts.

Hitchcock is closer associated to words like nightmare, which likely relates to his work in horror movies. While Anderson is closer to the word heart, reflecting his heartwarming style.